In [2]:
import requests
import requests.auth
import regex as re
from os.path  import basename
import random
import string
import json


client_auth = requests.auth.HTTPBasicAuth('AlFmsJweahucqYg4PwnQhQ', 'TiGAXDkqpxFKi9OYqTJiodHMGVt3Ew')
post_data = {"grant_type": "password", "username": "RedditorInCh1ef", "password": 'Redd1tPassw0rd'}
headers = {"User-Agent": "ChangeMeClient/0.1 by RedditorInCh1ef"}
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
auth_response = response.json()



In [5]:
def useable_link(i):
    """
    helper for get_links_from_post()
    """
    test = ['jpg','jpeg','png','imgur']
    for j in test:
        if j in i:
            return True
    return False

def get_links_from_post(postid):
    """
    Takes a post id and returns the possible img links
    """
    print(postid)
    comments = requests.get(f'http://oauth.reddit.com/r/photoshoprequest/comments/{postid}', headers=headers)
    c = comments.json()
    try:
        post_comments = [x['data']['body'] for x in c[1]['data']['children']] ##c[0] is post data it appears
        reg_link_find = 'https.+?(?=[)\n\]])'
        results = list(set([j for i in post_comments for j in re.findall(reg_link_find,i)]))
        results = [i for i in results if 'paypal' not in i] #no tipjar links
        results = [i for i in results if useable_link(i) ] #good links
        results = [i.replace('https://imgur.com/','https://i.imgur.com/').strip() for i in results]  #fix the imgur links these need file ext.
        results = [i + '.jpg' if ('imgur' in i) else i for i in results]
        return results
    except:
        return []

def get_random_string(length):
    # choose from all lowercase letter
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

def download_image(img_url,length_of_file = 20):
    
    try:
        res=requests.get(img_url,allow_redirects = True,headers= {'Content-type': 'content_type_value'})
        response = requests.get(img_url)
        ext = response.headers['Content-Type'].split('/')[1]
        img_bytes= response.content # download bytes for a image
        rand_string = get_random_string(length_of_file)
        file_name = f"{rand_string}.{ext}"
        if ext in ('jpeg','png'):
            with open(file_name,"wb") as img_file:
                img_file.write(img_bytes)
            return file_name
        else:
            pass
    except Exception as e:
        pass

def save_and_document(pre_img_url,post_image_urls,caption):
    """
    Takes a candidate pre_img_url and the resultant list from get_links_from_post
    saves the pre img if there is at least one output img.  saves files (makes sure they are unique)
    and adds to the metadata file.
    """
    output_image_list = []
    #input_image = [] ## this will be a list, of the ones that work.
    
    for link_candidate in post_image_urls:
        file_name = download_image(link_candidate)
        if file_name is not None:
            output_image_list.append(file_name)
    
    if len(output_image_list) > 0:
        for output_image in output_image_list:
            dictionary = {"file_name": output_image, "text":caption}
            json_object = json.dumps(dictionary, indent=4)
            with open("metadata.jsonl", "a") as outfile:
                outfile.write(json_object)
        #input_image_file = download_image(pre_img_url,length_of_file = 20)
    
        #if input_image_file is not None:
            #if len(input_image_file) > 0:
                for output_image in output_image_list:
                    dictionary = {"file_name": output_image, "text":caption}
                    json_object = json.dumps(dictionary, indent=4)
                    with open("metadata.jsonl", "a") as outfile:
                        outfile.write(json_object)
            
    #return input_image_file

In [6]:
token = auth_response['access_token']
auth = "bearer " + token
auth

'bearer 38076669--UdaSz9AP6tzfaTx1HNjsL5qIirRFw'

In [7]:
#t values hour, day, week, month, year, all
headers = {"Authorization": auth, "User-Agent": "testing still"}
response = requests.get("https://oauth.reddit.com/r/photoshoprequest/top/?t=year&limit=500", headers=headers)
x = response.json()

In [8]:
def posts_to_dataset(post_response):
    posts = post_response['data']['children']
    
    for post in posts:
        caption = post['data']['title']
        input_url = post['data']['url']
        postid = post['data']['id']
        print(post['data']['num_comments'])
        print(input_url)
        post_image_urls = get_links_from_post(postid)
        
        save_and_document(input_url,post_image_urls,caption)

In [9]:
posts_to_dataset(x)

203
https://i.redd.it/9ifaeov2yt991.jpg
vs3x5a
110
https://www.reddit.com/gallery/wnxe62
wnxe62
111
https://i.redd.it/2yj7fp1nkmd81.jpg
sbklfu
104
https://i.redd.it/pctynyao1t891.png
voftnr
114
https://i.redd.it/xl0xh2tkejf91.jpg
wfda34
127
https://i.redd.it/s07hk2jq66z91.jpg
yrfkcy
121
https://i.redd.it/1bofvpcd9hh81.jpg
sr2ujq
122
https://i.redd.it/zy1qpwxj5gg81.jpg
smvtdi
170
https://i.redd.it/cvewfcy4h2q91.jpg
xnyj15
147
https://i.redd.it/8anx3v1rgnp91.jpg
xm4z7v
104
https://i.redd.it/co5vjrul7f0a1.jpg
yx6h8z
140
https://i.redd.it/dm6gd9rdgiy91.png
yojh49
79
https://i.redd.it/detdvsu1xoa81.jpg
rzvj61
136
https://i.redd.it/18zo7qu8hss91.jpg
xzmtiv
82
https://i.redd.it/qcewlrn4eqi91.jpg
wsody1
121
https://www.reddit.com/gallery/y06a1j
y06a1j
224
https://i.redd.it/gpeo7t1dxet81.jpg
u37w8z
190
https://i.redd.it/pxbvj8fmpuk81.jpg
t4kpmv
88
https://i.redd.it/t25z9u5omco81.jpg
thw5z9
135
https://i.redd.it/oaurrxexwuj91.jpg
wxdi1k
116
https://i.redd.it/oyqvez4xkq881.jpg
rsasnx
136
https://